# Day 24

[Day 24 description](https://adventofcode.com/2018/day/24)

This is again a game modeling. Nothing particulary tricky, just take care of all rules. The only point of attention is for the second part: before having the immune system winning, we have a situation of stale (the battle never ends); once we detect this situation, the second part is just a binary search

In [1]:
import re

In [2]:
def to_int(xs):
    return [int(x) for x in xs]

In [3]:
raws = """
Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
 """[1:-1].split("\n")
n_infections = 2
n_immunes = 2

In [4]:
with open('AOC2018_24_input.txt') as f:
    raws = [x.strip() for x in f.readlines()]
n_infections = 10
n_immunes = 10

In [5]:
re_numbers = re.compile("""(\d+) units each with (\d+) hit points (?:\(.*\) )?with an attack that does (\d+) (?:\w+) damage at initiative (\d+)""")
re_atype = re.compile("""attack that does (?:\d+) (\w+) damage""")
re_weak = re.compile("""weak to ([^\);]*)""")
re_immune = re.compile("""immune to ([^\);]*)""")

In [6]:
class Group:
    def __init__(self, t, i, desc, boost=0):
        self.id = "%s_%s" % (t, i)
        n, hp, a, initiative = to_int(re_numbers.findall(desc)[0])
        weaknesses_ = re_weak.findall(desc)
        immunities_ = re_immune.findall(desc)
        atype_ = re_atype.findall(desc)
        self.n = n
        self.hp = hp
        self.a = a
        if t == 'imm':
            self.a += boost
        self.atype = atype_[0]
        self.initiative = initiative
        self.weaknesses = []
        if len(weaknesses_) > 0:
            self.weaknesses = [x.strip() for x in weaknesses_[0].split(",")]
        self.immunities = []
        if len(immunities_) > 0:
            self.immunities = [x.strip() for x in immunities_[0].split(",")]
    def power(self):
        return self.n * self.a
            
    def damage(self, other):
        base_damage = self.power()
        if self.atype in other.immunities:
            return 0
        elif self.atype in other.weaknesses:
            return 2*base_damage
        else:
            return base_damage

    def attack(self, other):
        dmg = self.damage(other)
        killed_units = dmg // other.hp
        other.n -= killed_units
        if other.n <= 0:
            return (True, killed_units)
        else:
            return (False, killed_units)
        
    def select_target(self, others):
        if len(others) == 0:
            return None
        max([(self.damage(x), x.power(), x.initiative) for x in others])
        d, _, _, target = max([(self.damage(x), x.power(), x.initiative, x) for x in others])
        if d > 0:
            return target
        else:
            return None
        
    def __repr__(self):
        return str(self.__dict__)

In [7]:
def compute_damage(g1, g2):
    base_damage = g1.power
    if g1.atype in g2.immunities:
        return 0
    if g1.atype in g2.weaknesses:
        return 2*base_damage
    else:
        return base_damage

In [8]:
### Immune system
immunes = {}
for i in range(1, 1+n_immunes):
    g = Group('imm', i, raws[i])
    immunes[g.id] = g

# Infection
infections = {}
for i in range(3 + n_immunes, 3 + n_immunes + n_infections):
    g = Group('inf', i-12, raws[i])
    infections[g.id] = g
    
all_groups = dict()
all_groups.update(immunes)
all_groups.update(infections)

In [9]:
c = 0
while len(infections) > 0 and len(immunes) > 0:
    c += 1
    list_of_attacks = []
    imm_taken = set()
    for id_inf, inf in sorted(infections.items(), key=lambda x: (-x[1].power(), -x[1].initiative)):
        targets = [v for k, v in immunes.items() if not k in imm_taken]
        enemy = inf.select_target(targets)
        if enemy:
            list_of_attacks.append((inf, enemy))
            imm_taken.add(enemy.id)
    inf_taken = set()
    for id_imm, imm in sorted(immunes.items(), key=lambda x: (-x[1].power(), -x[1].initiative)):
        targets = [v for k, v in infections.items() if not k in inf_taken]
        enemy = imm.select_target(targets)
        if enemy:
            list_of_attacks.append((imm, enemy))
            inf_taken.add(enemy.id)

    for attacker, defender in sorted(list_of_attacks, key=lambda x: -x[0].initiative):
        if attacker.n > 0:
            is_dead, k = attacker.attack(defender)
            if is_dead:
                if defender.id.startswith('imm'):
                    immunes.pop(defender.id)
                else:
                    infections.pop(defender.id)
            else:
                pass
print(sum([inf.n for k, inf in infections.items()]) + sum([inf.n for k, inf in immunes.items()]))

19381


In [10]:
boost = 34

### Immune system
immunes = {}
for i in range(1, 1+n_immunes):
    g = Group('imm', i, raws[i], boost)
    immunes[g.id] = g

# Infection
infections = {}
for i in range(3 + n_immunes, 3 + n_immunes + n_infections):
    g = Group('inf', i-12, raws[i])
    infections[g.id] = g
    
all_groups = dict()
all_groups.update(immunes)
all_groups.update(infections)

c = 0
while len(infections) > 0 and len(immunes) > 0:
    c += 1
    list_of_attacks = []
    imm_taken = set()
    for id_inf, inf in sorted(infections.items(), key=lambda x: (-x[1].power(), -x[1].initiative)):
        targets = [v for k, v in immunes.items() if not k in imm_taken]
        enemy = inf.select_target(targets)
        if enemy:
            list_of_attacks.append((inf, enemy))
            imm_taken.add(enemy.id)
    inf_taken = set()
    for id_imm, imm in sorted(immunes.items(), key=lambda x: (-x[1].power(), -x[1].initiative)):
        targets = [v for k, v in infections.items() if not k in inf_taken]
        enemy = imm.select_target(targets)
        if enemy:
            list_of_attacks.append((imm, enemy))
            inf_taken.add(enemy.id)

    for attacker, defender in sorted(list_of_attacks, key=lambda x: -x[0].initiative):
        if attacker.n > 0:
            is_dead, k = attacker.attack(defender)
            if is_dead:
                if defender.id.startswith('imm'):
                    immunes.pop(defender.id)
                else:
                    infections.pop(defender.id)
            else:
                pass
print(sum([inf.n for k, inf in infections.items()]) + sum([inf.n for k, inf in immunes.items()]))

3045
